# ***Precio Promedio por M2 cuadrados***

In [2]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "alfa_q" in nombre and "pachuca" in nombre

    #if nombre.startswith("df_") and "tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_q_feb_2023_pachuca',
 'df_alfa_q_jul_2023_pachuca',
 'df_alfa_q_jun_2023_pachuca',
 'df_alfa_q_mar_2023_pachuca',
 'df_alfa_q_nov_2022_pachuca',
 'df_alfa_q_oct_2022_pachuca',
 'df_alfa_q_sep_2023_pachuca']

In [9]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'precio_m2_contruido':'precio_m2_construido','id_jul_23':'id','m2_contruido': 'm2_construido','m_construido': 'm2_construido','Metros_construido':'m2_construido','segmento':'categoria','Category':'categoria','m_total':'m2_total','m_construido':'m2_construido'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [10]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['precio', 'm2_total','m2_construido', 'precio_m2_construido', 'precio_m2_terreno']]
    # Añadir el DataFrame a la listaprecio_m2_construido
    dataframes_list.append(segment_df)

### *DF CON RANGOS DE M2 Y SU PRECIO*

In [11]:
new_dataframes_list = []

# Definir los límites de los rangos
rangos = [0, 100, 150, 200, 250, 300, float('inf')]
labels = ['[0-100]', '[100-150]', '[150-200]', '[200-250]', '[250-300]', '[> 300]']

# Iterar sobre cada DataFrame en la lista original
for df in dataframes_list:
    # Crear una copia del DataFrame
    modified_df = df.copy()
    # Asignar el rango correspondiente a cada valor de 'm2_construido'
    modified_df['rango'] = pd.cut(modified_df['m2_total'], bins=rangos, labels=labels, right=False)
    # Crear un nuevo DataFrame solo con las columnas 'rango' y 'precio_m2_terreno'
    new_df = modified_df[['rango', 'precio_m2_terreno']].copy()
    # Agregar el nuevo DataFrame a la lista
    new_dataframes_list.append(new_df)

# Imprimir los primeros registros de cada nuevo DataFrame
for idx, df in enumerate(new_dataframes_list):
    print(f"Primeros registros de nuevo DataFrame {idx + 1}:")
    print(df.head())
    print()  # Agregar una línea en blanco entre cada DataFrame


Primeros registros de nuevo DataFrame 1:
     rango  precio_m2_terreno
0  [0-100]               8896
1  [0-100]              12672
2  [0-100]              11328
3  [0-100]              12239
4  [0-100]              12791

Primeros registros de nuevo DataFrame 2:
     rango  precio_m2_terreno
0  [0-100]           13055.56
1  [0-100]           12589.29
2  [0-100]           26754.08
3  [0-100]           28196.72
4  [0-100]           28524.59

Primeros registros de nuevo DataFrame 3:
     rango  precio_m2_terreno
0  [0-100]           33333.33
1  [0-100]           16688.89
2  [0-100]           35714.29
3  [0-100]          221000.00
4  [0-100]           12277.78

Primeros registros de nuevo DataFrame 4:
     rango  precio_m2_terreno
0  [0-100]           13550.98
1  [0-100]            7037.04
2  [0-100]           12054.55
3  [0-100]            8166.67
4  [0-100]           11616.67

Primeros registros de nuevo DataFrame 5:
     rango  precio_m2_terreno
0  [0-100]           12777.78
1  [0-100] 

### *Promedio de precio_m2_terreno por rango CADA DF* 

In [12]:
# Iterar sobre cada DataFrame modificado en new_dataframes_list
for idx, df in enumerate(new_dataframes_list):
    print(f"Promedio de precio_m2_terreno por rango en el DataFrame {idx + 1}:")
    # Agrupar el DataFrame por la columna 'rango' y calcular el promedio de 'precio_m2_terreno' para cada grupo
    promedios_por_rango = df.groupby('rango')['precio_m2_terreno'].mean()
    print(promedios_por_rango)
    print()  # Agregar una línea en blanco entre cada resultado

Promedio de precio_m2_terreno por rango en el DataFrame 1:
rango
[0-100]      15382.663717
[100-150]    19318.665992
[150-200]    21102.187500
[200-250]    20318.400000
[250-300]    16719.153846
[> 300]      15391.483871
Name: precio_m2_terreno, dtype: float64

Promedio de precio_m2_terreno por rango en el DataFrame 2:
rango
[0-100]      23219.324524
[100-150]    23866.457742
[150-200]    15643.237895
[200-250]    14384.869545
[250-300]    15627.686154
[> 300]      16863.140000
Name: precio_m2_terreno, dtype: float64

Promedio de precio_m2_terreno por rango en el DataFrame 3:
rango
[0-100]      18465.115040
[100-150]    19280.987941
[150-200]    22039.477255
[200-250]    22622.139507
[250-300]    25955.239111
[> 300]      21078.173986
Name: precio_m2_terreno, dtype: float64

Promedio de precio_m2_terreno por rango en el DataFrame 4:
rango
[0-100]      16374.213392
[100-150]    18753.976184
[150-200]    20514.648247
[200-250]    21956.151608
[250-300]    20120.904638
[> 300]      19474.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_11356\2294499642.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  promedios_por_rango = df.groupby('rango')['precio_m2_terreno'].mean()


In [13]:
# Inicializar un diccionario para almacenar los promedios de precio por metro cuadrado para cada rango
promedios_por_rango_dict = {}
# Iterar sobre cada DataFrame modificado en new_dataframes_list
for idx, df in enumerate(new_dataframes_list):
    # Agrupar el DataFrame por la columna 'rango' y calcular el promedio de 'precio_m2_terreno' para cada grupo
    promedios_por_rango = df.groupby('rango')['precio_m2_terreno'].mean()
    # Actualizar el diccionario con los promedios calculados
    promedios_por_rango_dict[f'DataFrame_{idx + 1}'] = promedios_por_rango

# Convertir el diccionario en un DataFrame
rango_precio_m2 = pd.DataFrame(promedios_por_rango_dict)
print("DataFrame 'rango_precio_m2':")
rango_precio_m2

DataFrame 'rango_precio_m2':


C:\Users\yoe11\AppData\Local\Temp\ipykernel_11356\225855980.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  promedios_por_rango = df.groupby('rango')['precio_m2_terreno'].mean()


,DataFrame_1,DataFrame_2,DataFrame_3,DataFrame_4,DataFrame_5,DataFrame_6,DataFrame_7
rango,,,,,,,
[0-100],15382.663717,23219.324524,18465.115040,16374.213392,16242.468095,16469.612659,18269.369817
[100-150],19318.665992,23866.457742,19280.987941,18753.976184,18512.567189,18763.317388,20672.039103
[150-200],21102.187500,15643.237895,22039.477255,20514.648247,21128.278100,21638.509925,22592.415630
[200-250],20318.400000,14384.869545,22622.139507,21956.151608,20652.793333,21694.308655,24542.754419
[250-300],16719.153846,15627.686154,25955.239111,20120.904638,23581.253864,25145.554103,24464.570000
[> 300],15391.483871,16863.140000,21078.173986,19474.847652,22931.931270,24058.407167,18427.718424


In [14]:
# Calcular el promedio general de cada rango en todas las columnas de rango_precio_m2
promedio_general_por_rango = rango_precio_m2.mean(axis=1)
# Crear un nuevo DataFrame 'Precio_mean_M2' con los rangos y sus promedios generales
Precio_mean_M2 = pd.DataFrame({'Rango': promedio_general_por_rango.index, 'Precio_m2_mean': promedio_general_por_rango.values})

print("DataFrame 'Precio_mean_M2':")
Precio_mean_M2

DataFrame 'Precio_mean_M2':


,Rango,Precio_m2_mean
0,[0-100],17774.681035
1,[100-150],19881.144506
2,[150-200],20665.536365
3,[200-250],20881.631010
4,[250-300],21659.194531
5,[> 300],19746.528910


# Gráfica

In [15]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#F79CB9']
porcentajes = Precio_mean_M2['Precio_m2_mean']

# Convertir la columna 'Precio_m2_mean' a números de punto flotante
#Precio_mean_M2['Precio_m2_mean'] = Precio_mean_M2['Precio_m2_mean'].str.replace('$', '').str.replace(',', '').astype(float)
# Convertir los valores a cadenas con el formato deseado
textos_formateados = [f'${round(valor, 2):,.2f}' for valor in Precio_mean_M2['Precio_m2_mean']]

fig = go.Figure()
# Obtener el valor máximo redondeado al próximo múltiplo de 5000
max_valor = int(Precio_mean_M2['Precio_m2_mean'].max() / 5000 + 1) * 5000

# Agregar los datos como barras
fig.add_trace(go.Bar(
    x=Precio_mean_M2['Rango'],  # Eje x: Modalidades
    y=Precio_mean_M2['Precio_m2_mean'],  # Eje y: Porcentajes
    marker_color=colores,  # Especifica los colores de las barras
    text=textos_formateados,  # Texto que se mostrará en las barras (porcentaje)
    textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
))

# Personalizar el diseño de la gráfica
fig.update_layout(
    #title='Precio promedio por M2 totales.',  # Título de la gráfica
    #xaxis=dict(title='Rango'),  # Título del eje x
    yaxis=dict(
        #title='Precio', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        gridwidth=1,
        tickvals=list(range(0, max_valor + 5000, 5000)),  # Valores de los ticks del eje y
        ticktext=[f'${valor:.2f}' for valor in range(0, max_valor + 5000, 5000)]  # Textos de los ticks del eje y
    ),  
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    height=500,  # Altura de la gráfica
    margin=dict(l=10, r=10, t=10, b=10)  # Ajusta los márgenes (left, right, top, bottom)

)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
guardar_grafico_como_html(fig, 'g_bar_PrecioPromedio_m2', carpeta='assets/graficas')
fig.show()